In [ ]:
import math as m
import numpy as np 
import pandas as pd
import pprint
from sklearn.feature_selection import chi2

In [ ]:
try:
    kordinate = pd.read_csv('Kordinate.csv')
    kordinate =kordinate.to_numpy()
except:
    print("Nisu ucitane kordinate")
try:
    pravci = pd.read_csv('Pravci.csv')
    pravci = pravci.to_numpy()
    pravci = pravci -1
except:
    pravci = np.empty(shape=(0,0))
try:
    duzine = pd.read_csv('Duzine.csv')
    duzine = duzine.to_numpy()
    duzine = duzine -1
except:
    duzine =np.empty(shape=(0,0))
try:
    uglovi = pd.read_csv('Uglovi.csv')
    uglovi.to_numpy()
    uglovi = uglovi-1
except:
    uglovi = np.empty(shape=(0,0))
ro = 180/m.pi*3600



In [ ]:
def direkcioni(y1,x1,y2,x2):
    deltaY = y2-y1
    deltaX = x2-x1 
    
    
    if deltaX > 0  and deltaY > 0:
        ugao = m.atan(abs(deltaY/deltaX))
    elif deltaX < 0 and deltaY > 0:
        ugao = m.atan(abs(deltaX/deltaY))+m.pi/2
    elif deltaX <0 and deltaY < 0:
        ugao = m.atan(abs(deltaY,deltaX))+ m.pi
    elif deltaX > 0 and deltaY < 0 :
        ugao = m.atan(abs(deltaX/deltaY))+(3/2)*m.pi
    elif deltaX == 0 and deltaY > 0:
        ugao = m.pi/2
    elif deltaX == 0 and deltaY < 0:
        ugao = m.pi*3/2
    elif deltaX > 0 and deltaY ==0:
        ugao = 0 
    elif deltaX < 0 and deltaY ==0:
        ugao = m.pi
    
    return ugao
    
def odstampajUgao(a):
    minuta = (a%1)*60
    sekunda =(minuta%1)*60
    return "{0}\u00b0 {1}\' {2}\"".format(int(a),int(minuta),int(sekunda))
    

In [ ]:
def direkcionik(kord1,kord2):
    deltaY = kord2[0]-kord1[0]
    deltaX = kord2[1]-kord1[1]
    ugao = 0
    
    if deltaX > 0  and deltaY > 0:
        ugao = m.atan(abs(deltaY/deltaX))
    elif deltaX < 0 and deltaY > 0:
        ugao = m.atan(abs(deltaX/deltaY))+m.pi/2
    elif deltaX <0 and deltaY < 0:
        ugao = m.atan(abs(deltaY/deltaX)) + m.pi
    elif deltaX > 0 and deltaY < 0 :
        ugao = m.atan(abs(deltaX/deltaY))+(3/2)*m.pi
    elif deltaX == 0 and deltaY > 0:
        ugao = m.pi/2
    elif deltaX == 0 and deltaY < 0:
        ugao = m.pi*3/2
    elif deltaX > 0 and deltaY ==0:
        ugao = 0 
    elif deltaX < 0 and deltaY ==0:
        ugao = m.pi
    
    return ugao

In [ ]:
def duzina(y1,x1,y2,x2):
    return m.sqrt(m.pow((y2-y1),2)+m.pow((x2-x1),2))
def duzinak(kord1,kord2):
    return m.sqrt(m.pow((kord2[0]-kord1[0]),2)+m.pow((kord2[1]-kord1[1]),2))


In [ ]:
def pravcif(stanica,vizura):
    diru = direkcionik(kordinate[stanica],kordinate[vizura])
    duzina = duzinak(kordinate[stanica],kordinate[vizura])
    aij = ro*m.sin(diru)/(duzina*1000)
    bij = -ro*m.cos(diru)/(duzina*1000)
    return aij,bij
    

def merened(stanica,vizura):
    diru = direkcionik(kordinate[stanica],kordinate[vizura])
    Aij = -m.cos(diru)
    Bij = -m.sin(diru)
    
    return Aij, Bij

In [ ]:
def pravcip(p):
    diru = direkcionik(kordinate[p[0]],kordinate[p[1]])
    duzina = duzinak(kordinate[p[0]],kordinate[p[1]])
    aij = ro*m.sin(diru)/(duzina*1000)
    bij = -ro*m.cos(diru)/(duzina*1000)
    return aij,bij
    
def merenedd(d):
    diru = direkcionik(kordinate[d[0]],kordinate[d[1]])
    Aij = -m.cos(diru)
    Bij = -m.sin(diru)
    
    return Aij, Bij

In [ ]:
duzinak(kordinate[6],kordinate[0])

In [ ]:
def kolonaAP(arr):
    kolonaA = np.zeros((1,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
    kolonaA[0,2*arr[0]],kolonaA[0,2*arr[0]+1] = pravcip(arr)[1],pravcip(arr)[0]
    kolonaA[0,2*arr[1]],kolonaA[0,2*arr[1]+1] = -kolonaA[0,2*arr[0]],-kolonaA[0,2*arr[0]+1]
    kolonaA[0,kordinate.size+arr[0]]=1
    return kolonaA
    
def kolonaAD(arr):
    kolonaA = np.zeros((1,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
    kolonaA[0,2*arr[0]],kolonaA[0,2*arr[0]+1] = merenedd(arr)[1],merenedd(arr)[0]
    kolonaA[0,2*arr[1]],kolonaA[0,2*arr[1]+1] = -kolonaA[0,2*arr[0]],-kolonaA[0,2*arr[0]+1]
    return kolonaA



In [ ]:
def kolonaAU(arr):
    kolonaA = np.zeros((1,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
    kolonaA[0,2*arr[0]]=pravcip(arr[[0,1]])[1] -pravcip(arr[[0,2]])[1]
    kolonaA[0,2*arr[0]+1]=pravcip(arr[[0,1]])[0]-pravcip(arr[[0,2]])[0]
    kolonaA[0,2*arr[1]]=pravcip(arr[[1,0]])[1]
    kolonaA[0,2*arr[1]+1]=pravcip(arr[[1,0]])[0]
    kolonaA[0,2*arr[2]]=-pravcip(arr[[2,0]])[1]
    kolonaA[0,2*arr[2]+1]=-pravcip(arr[[2,0]])[0]
    
    return kolonaA

In [ ]:
def matricaA(pravci,duzine,uglovi):
    
    if pravci.size and not duzine.size and not uglovi.size:
        arrays1 = [kolonaAP(i) for i in pravci] 
        A = np.vstack(tuple(arrays1)) 
        
    elif duzine.size and not pravci.size and not uglovi.size:
        arrays2 = [kolonaAD(i) for i in duzine]
        A=np.vstack(tuple(arrays2))  
    
    elif uglovi.size and not duzine.size and not pravci.size:
        arrays3 = [kolonaAU(i) for i in uglovi]
        A=np.vstack(tuple(arrays3)) 
        
    elif pravci.size and duzine.size and not uglovi.size:
        arrays1 = [kolonaAP(i) for i in pravci] 
        P = np.vstack(tuple(arrays1))
        arrays2 = [kolonaAD(i) for i in duzine]
        D=np.vstack(tuple(arrays2))
        A = np.vstack((P,D))
    elif pravci.size and not duzine.size and uglovi.size:
        arrays1 = [kolonaAP(i) for i in pravci] 
        P = np.vstack(tuple(arrays1))
        arrays3 = [kolonaAU(i) for i in uglovi]
        U=np.vstack(tuple(arrays3)) 
        A = np.vstack((P,U))
    elif not pravci.shape and duzine.size and uglovi.size:
        arrays3 = [kolonaAU(i) for i in uglovi]
        U=np.vstack(tuple(arrays3)) 
        arrays2 = [kolonaAD(i) for i in duzine]
        D=np.vstack(tuple(arrays2))
        A = np.vstack((D,U))
    elif pravci.shape and duzine.size and uglovi.size:
        arrays1 = [kolonaAP(i) for i in pravci] 
        P = np.vstack(tuple(arrays1))
        arrays3 = [kolonaAU(i) for i in uglovi]
        U=np.vstack(tuple(arrays3)) 
        arrays2 = [kolonaAD(i) for i in duzine]
        D=np.vstack(tuple(arrays2))
        A = np.vstack((P,D,U))
        
        
        
    return A
A = matricaA(pravci,duzine,uglovi)


In [ ]:
br_girusa = 1
br_merenja =3
sigma_p = 2
sigma_d = [3,3]
sigma = 1 

In [ ]:
def p_alpha(sigma_p,br_girusa,sigma,pravci):
    sigma_a = sigma_p/m.sqrt(br_girusa)
    p_alpha = np.array([sigma/sigma_a**2 for i in range(pravci.shape[0])])
    return p_alpha
a = p_alpha(sigma_p,br_girusa,sigma,pravci)
a


In [ ]:
def p_duzina(br_merenja,sigma_d,sigma,kordinate):
    duzina = []
    for i in duzine:
        duzina.append(duzinak(kordinate[i[0]],kordinate[i[1]]))
    duzina = np.array(duzina)
    d_sigma = np.array([1,2])
    d_sigma = sigma_d[0]+sigma_d[1]*(duzina/1000)
    d_sigma = d_sigma/m.sqrt(br_merenja)
    
    p_duzina = sigma/d_sigma**2

    return p_duzina

a=p_duzina(br_merenja,sigma_d,sigma,kordinate)
a.round(2)

In [ ]:
def p_uglova(sigma_p,br_girusa,sigma,uglovi):
    sigma_a = sigma_p/m.sqrt(br_girusa)
    p_alpha = np.array([sigma/sigma_a**2 for i in range(uglovi.shape[0])])
    return p_alpha*m.sqrt(2)

In [ ]:
def matricaP(sigma_p,br_girusa,sigma,pravci,br_merenja,sigma_d,kordinate,uglovi):
    p_a = p_alpha(sigma_p,br_girusa,sigma,pravci)
    p_d=p_duzina(br_merenja,sigma_d,sigma,kordinate)
    p_u = p_uglova(sigma_p,br_girusa,sigma,uglovi)
    p=np.append(p_a,p_d)
    p = np.append(p,p_u)
    P = np.diag(p)
    return P
    
A.shape

In [ ]:
P = matricaP(sigma_p,br_girusa,sigma,pravci,br_merenja,sigma_d,kordinate,uglovi)
np.diag(P)
P.shape

In [ ]:
N = np.matmul(np.matmul(A.transpose(),P),A)



## Analiza tacnosti

In [ ]:
tacka1 = 'y5'
tacka2 = 'x5'
tacka3 ='y2'



In [ ]:
def indexRT(s):
    br = 0
    if s[0].upper() =="Y":
        var = 0
    elif s[0].upper() == "X":
        var = 1
    else:
        return "Pogresno ste uneli kordinatu"
    try:
        br = int(s[1]) - 1 
    except:
        print("Pogresno ste uneli kordinatu")
    
    return 2*br + var

In [ ]:
def matricaR(tacka1,tacka2,tacka3):
    R=np.zeros((3,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
    R[0][indexRT(tacka1)]=1
    R[1][indexRT(tacka2)]=1
    R[2][indexRT(tacka3)]=1
    return R

In [ ]:
R = matricaR(tacka1,tacka2,tacka3)


In [ ]:
Nplus =np.hstack((np.vstack((N,R)),np.vstack((R.transpose(),np.zeros((R.shape[0],R.shape[0]))))))
Nplus.shape


In [ ]:
def matricaN(A,P,RB):
    N = np.matmul(np.matmul(A.transpose(),P),A)
    N =np.hstack((np.vstack((N,RB)),np.vstack((RB.transpose(),np.zeros((RB.shape[0],RB.shape[0]))))))
    return N
    

In [ ]:
N = matricaN(A,P,R)


In [ ]:
def kv(Qxy,Qxx,Qyy):
    var1 = 2*Qxy
    var2 = Qxx-Qyy
    k=0
    K = np.zeros(var1.shape[0])
    for i in range(var1.shape[0]):

        if var1[i] > 0 and var2[i] >0:
            k = 0
        elif var1[i] >0 and var2[i] <0:
            k = m.pi
        elif var1[i] < 0 and var2[i] < 0:
            k =m.pi
        elif var1[i] < 0 and var2[i] >0:
            k = 2*m.pi
        elif var1[i] ==0:
            k=0
        K[i]=k

    return K

In [ ]:
def analizaTacnosti(A,P,N,RB):
    Qx = np.linalg.inv(N)
    Qx_kap = Qx[:-RB.shape[0] ,:-RB.shape[0]]
    Qx_kap = np.round(Qx_kap,decimals=5)
    Qi = np.matmul(np.matmul(A,Qx_kap),A.transpose())
    Qv = np.linalg.inv(P) - Qi
    Qxi = np.diag(Qx_kap)
    Qxi = Qxi[:kordinate.size]
    Qyy = Qxi[0::2]
    Qyy = np.round(Qyy,decimals = 3)
    Qxx = Qxi[1::2]
    Qxx = np.round(Qxx,decimals = 3)
    Qxy = np.diag(Qx_kap,k=1)[:kordinate.size][::2]
    sigma_y = sigma*np.sqrt(Qyy)
    sigma_x = sigma*np.sqrt(Qxx)
    sigma_polozaja = np.sqrt(sigma_y**2 + sigma_x**2)
    k = np.sqrt((Qxx-Qyy)**2 +4*Qxy**2)
    lamda1 = 0.5*(Qxx+Qyy + k)
    lamda2 = 0.5*(Qxx+Qyy-k)
    chi = 5.99146454710798
    Ai = sigma*np.sqrt(lamda1*chi)
    Bi = sigma*np.sqrt(lamda2*chi)
    KV = kv(Qxy,Qxx,Qyy)
    theta = 0.5*(np.arctan(2*Qxy/(Qxx-Qyy))+KV)
    theta[np.isnan(theta)]=0
    np.degrees(theta)
    theta_deg = np.degrees(theta)
    
    return Qx,Qi,Qv,Qxi,Qyy,Qxx,Qxy,sigma_y,sigma_x,sigma_polozaja,k,lamda1,lamda2,Ai,Bi,theta,theta_deg

In [ ]:
Qx,Qi,Qv,Qxi,Qyy,Qxx,Qxy,sigma_y,sigma_x,sigma_polozaja,k,lamda1,lamda2,Ai,Bi,theta,theta_deg = analizaTacnosti(A,P,N,R)

## Unutrasnja pouzdanost


In [ ]:
def unustarsnjaPouzdanost(Qv,P):
    Qvi = np.diag(Qv)
    Pi=np.diag(P)
    ri = Qvi*Pi
    Gi =2.802/(Pi*np.sqrt(Qvi))
    return ri,Gi
ri,Gi = unustarsnjaPouzdanost(Qv,P)


## Zadatak pod B

In [ ]:
def matricaB(kordinate,pravci,duzine):
    x0=np.average(kordinate[:,1])
    y0=np.average(kordinate[:,0])
    g = m.sqrt(np.sum((kordinate[:,1]-x0)**2) + np.sum((kordinate[:,0]-y0)**2))
    yeta =(kordinate[:,0]-y0)/g
    xeta =(kordinate[:,1]-x0)/g 
    Bt = np.zeros((3,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
    var = 1/(m.sqrt(kordinate.shape[0]))
    Bt[0,0:kordinate.size:2]=var
    Bt[1,1:kordinate.size:2]=var
    Bt[2,:kordinate.size:2]=-xeta
    Bt[2,1:kordinate.size:2]=yeta
    if not duzine.size:
        bt4 = np.zeros((1,kordinate.size+len(np.unique(pravci[:,0]))),dtype = float)
        bt4[0,:kordinate.size:2]=yeta
        bt4[0,1:kordinate.size:2]=xeta
        Bt = np.vstack((Bt,bt4))
    return Bt
  

In [ ]:
B = matricaB(kordinate,pravci,duzine)
bN = matricaN(A,P,B)


In [ ]:
bQx,bQi,bQv,bQxi,bQyy,bQxx,bQxy,bsigma_y,bsigma_x,bsigma_polozaja,bk,blamda1,blamda2,bAi,bBi,btheta,btheta_deg = analizaTacnosti(A,P,bN,B)

In [ ]:
bri,bGi=unustarsnjaPouzdanost(bQv,P)

In [ ]:
Af = pd.DataFrame(A.round(3))
Pf = pd.DataFrame(P.round(3))
Nf = pd.DataFrame(N.round(3))
Rf = pd.DataFrame(R.round(3))
Qxf = pd.DataFrame(Qx.round(3))
Qif = pd.DataFrame(Qi.round(3))
Qvf = pd.DataFrame(Qv.round(3))
Qxif = pd.DataFrame(Qxi.round(3))
Qyyf = pd.DataFrame(Qyy.round(3))
Qxxf = pd.DataFrame(Qxx.round(3))
Qxyf = pd.DataFrame(Qxy.round(3))
sigma_yf = pd.DataFrame(sigma_y.round(3))
sigma_xf = pd.DataFrame(sigma_x.round(3))
sigma_polozajaf= pd.DataFrame(sigma_polozaja.round(3))
lamda1f = pd.DataFrame(lamda1.round(3))
lamda2f= pd.DataFrame(lamda2.round(3))
Aif = pd.DataFrame(Ai.round(3))
Bif = pd.DataFrame(Bi.round(3))
thetaf = pd.DataFrame(theta.round(3))
theta_degf = pd.DataFrame(theta_deg.round(3))
rif = pd.DataFrame(ri.round(3))
Gif = pd.DataFrame(Gi.round(3))

Btf = pd.DataFrame(B.round(3))
bNf = pd.DataFrame(bN.round(3))
bQxf = pd.DataFrame(bQx.round(3))
bQif = pd.DataFrame(bQi.round(3))
bQvf = pd.DataFrame(bQv.round(3))
bQxif = pd.DataFrame(bQxi.round(3))
bQyyf = pd.DataFrame(bQyy.round(3))
bQxxf = pd.DataFrame(bQxx.round(3))
bQxyf = pd.DataFrame(bQxy.round(3))
bsigma_yf = pd.DataFrame(bsigma_y.round(3))
bsigma_xf = pd.DataFrame(bsigma_x.round(3))
bsigma_polozajaf= pd.DataFrame(bsigma_polozaja.round(3))
blamda1f = pd.DataFrame(blamda1.round(3))
blamda2f= pd.DataFrame(blamda2.round(3))
bAif = pd.DataFrame(bAi.round(3))
bBif = pd.DataFrame(bBi.round(3))
bthetaf = pd.DataFrame(btheta.round(3))
btheta_degf = pd.DataFrame(btheta_deg.round(3))
brif = pd.DataFrame(bri.round(3))
bGif = pd.DataFrame(bGi.round(3))


In [ ]:
dfs = {"Matrica A":Af,"Matrica P": Pf ,"Matrica N":Nf,"Matrica R":Rf, "Matrica Qx":Qxf,"Matrica Qi" : Qif,"Matrica Qv":Qvf,
      "Matrica Qxi":Qxif,"Matrica Qyy":Qyyf,"Matrica Qxx":Qxxf,"Matrica Qxy":Qxyf," Sigma y":sigma_yf,"Sigma x":sigma_xf,
       "Sigma polozaja":sigma_polozajaf,"Lamda 1":lamda1f,"Lamda 2":lamda2f,"Elementi Ai":Aif,"Elementi Bi":Bif,"Teta":thetaf,
       "Elementi ri":rif,"Elementi Gi":Gif,"Zadatak pod b ":pd.DataFrame(),"Matrica Bt":Btf, "Matrica bN":bNf, "Matrica bQx":bQxf,
       "Matrica bQi" : bQif,"Matrica bQv":bQvf,
       "Matrica bQxi":bQxif,"Matrica bQyy":bQyyf,"Matrica bQxx":bQxxf,"Matrica bQxy":bQxyf," Sigma by":bsigma_yf,"Sigma bx":bsigma_xf,
       "Sigma polozajab":bsigma_polozajaf,"Lamda b1":blamda1f,"Lamda b2":blamda2f,
       "Elementi bAi":bAif,"Elementi bBi":bBif,"Tetab":bthetaf,"Elementi bri":brif,"Elementi bGi":bGif
      }

writer = pd.ExcelWriter('resenja.xlsx', engine='xlsxwriter')
for sheet_name in dfs.keys():
    dfs[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False,header = False)
    
writer.save()

In [ ]:
def resenje(kordinate,pravci,duzine,uglovi,br_girusa,br_merenja,sigma_p,sigma_d,sigma,tacka1,tacka2,tacka3):
    
    A = matricaA(pravci,duzine,uglovi)
    P = matricaP(sigma_p,br_girusa,sigma,pravci,br_merenja,sigma_d,kordinate,uglovi)
    R = matricaR(tacka1,tacka2,tacka3)
    N = matricaN(A,P,R)
    Qx,Qi,Qv,Qxi,Qyy,Qxx,Qxy,sigma_y,sigma_x,sigma_polozaja,k,lamda1,lamda2,Ai,Bi,theta,theta_deg = analizaTacnosti(A,P,N,R)
    ri,Gi = unustarsnjaPouzdanost(Qv,P)
    B = matricaB(kordinate,pravci,duzine)
    bN = matricaN(A,P,B)
    bQx,bQi,bQv,bQxi,bQyy,bQxx,bQxy,bsigma_y,bsigma_x,bsigma_polozaja,bk,blamda1,blamda2,bAi,bBi,btheta,btheta_deg = analizaTacnosti(A,P,bN,B)
    bri,bGi=unustarsnjaPouzdanost(bQv,P)
    
    Af = pd.DataFrame(A.round(3))
    Pf = pd.DataFrame(P.round(3))
    Nf = pd.DataFrame(N.round(3))
    Rf = pd.DataFrame(R.round(3))
    Qxf = pd.DataFrame(Qx.round(3))
    Qif = pd.DataFrame(Qi.round(3))
    Qvf = pd.DataFrame(Qv.round(3))
    Qxif = pd.DataFrame(Qxi.round(3))
    Qyyf = pd.DataFrame(Qyy.round(3))
    Qxxf = pd.DataFrame(Qxx.round(3))
    Qxyf = pd.DataFrame(Qxy.round(3))
    sigma_yf = pd.DataFrame(sigma_y.round(3))
    sigma_xf = pd.DataFrame(sigma_x.round(3))
    sigma_polozajaf= pd.DataFrame(sigma_polozaja.round(3))
    lamda1f = pd.DataFrame(lamda1.round(3))
    lamda2f= pd.DataFrame(lamda2.round(3))
    Aif = pd.DataFrame(Ai.round(3))
    Bif = pd.DataFrame(Bi.round(3))
    thetaf = pd.DataFrame(theta.round(3))
    theta_degf = pd.DataFrame(theta_deg.round(3))
    rif = pd.DataFrame(ri.round(3))
    Gif = pd.DataFrame(Gi.round(3))

    Btf = pd.DataFrame(B.round(3))
    bNf = pd.DataFrame(bN.round(3))
    bQxf = pd.DataFrame(bQx.round(3))
    bQif = pd.DataFrame(bQi.round(3))
    bQvf = pd.DataFrame(bQv.round(3))
    bQxif = pd.DataFrame(bQxi.round(3))
    bQyyf = pd.DataFrame(bQyy.round(3))
    bQxxf = pd.DataFrame(bQxx.round(3))
    bQxyf = pd.DataFrame(bQxy.round(3))
    bsigma_yf = pd.DataFrame(bsigma_y.round(3))
    bsigma_xf = pd.DataFrame(bsigma_x.round(3))
    bsigma_polozajaf= pd.DataFrame(bsigma_polozaja.round(3))
    blamda1f = pd.DataFrame(blamda1.round(3))
    blamda2f= pd.DataFrame(blamda2.round(3))
    bAif = pd.DataFrame(bAi.round(3))
    bBif = pd.DataFrame(bBi.round(3))
    bthetaf = pd.DataFrame(btheta.round(3))
    btheta_degf = pd.DataFrame(btheta_deg.round(3))
    brif = pd.DataFrame(bri.round(3))
    bGif = pd.DataFrame(bGi.round(3))


    dfs = {"Matrica A":Af,"Matrica P": Pf ,"Matrica N":Nf,"Matrica R":Rf, "Matrica Qx":Qxf,"Matrica Qi" : Qif,"Matrica Qv":Qvf,
      "Matrica Qxi":Qxif,"Matrica Qyy":Qyyf,"Matrica Qxx":Qxxf,"Matrica Qxy":Qxyf," Sigma y":sigma_yf,"Sigma x":sigma_xf,
       "Sigma polozaja":sigma_polozajaf,"Lamda 1":lamda1f,"Lamda 2":lamda2f,"Elementi Ai":Aif,"Elementi Bi":Bif,"Teta":thetaf,
       "Elementi ri":rif,"Elementi Gi":Gif,"Zadatak pod b ":pd.DataFrame(),"Matrica Bt":Btf, "Matrica bN":bNf, "Matrica bQx":bQxf,
       "Matrica bQi" : bQif,"Matrica bQv":bQvf,
       "Matrica bQxi":bQxif,"Matrica bQyy":bQyyf,"Matrica bQxx":bQxxf,"Matrica bQxy":bQxyf," Sigma by":bsigma_yf,"Sigma bx":bsigma_xf,
       "Sigma polozajab":bsigma_polozajaf,"Lamda b1":blamda1f,"Lamda b2":blamda2f,
       "Elementi bAi":bAif,"Elementi bBi":bBif,"Tetab":bthetaf,"Elementi bri":brif,"Elementi bGi":bGif
      }

    writer = pd.ExcelWriter('resenja.xlsx', engine='xlsxwriter')
    for sheet_name in dfs.keys():
        dfs[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False,header = False)

    writer.save()
    
    